In [ ]:
!pip install sentence-transformers cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.3 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from google.colab import userdata
import pickle
import cohere
import json

# Initialize the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Load the embeddings and product names
with open('embeddings_and_products.pkl', 'rb') as file:
    saved_data = pickle.load(file)

name_embeddings = saved_data['embeddings']  # Loaded embeddings
product_names = saved_data['products']  # Loaded product names

In [ ]:
# Define a function for semantic search
def semantic_search(query, embeddings, items, top_n=1):
    query_embedding = model.encode(query)
    similarities = cosine_similarity([query_embedding], embeddings)[0]
    top_indices = np.argsort(similarities)[::-1][:top_n]
    #return [(items[i], similarities[i]) for i in top_indices]
    return [items[i] for i in top_indices]

In [ ]:
# Test the semantic search
query = "Ribs"
search_results = semantic_search(query, name_embeddings, product_names)
print(search_results)

# #Display results
# for result in search_results:
#     print(f"Product: {result[0]}, Similarity: {result[1]:.4f}")

['Short Ribs, Boneless']


In [ ]:
from google.colab import userdata
cohere_api_key = userdata.get('COHERE_API_KEY')

In [ ]:
def construct_prompt(query):

    prompt = f'''
    As an AI assistant for an online grocery shopping platform, your role is to generate discovery-oriented shopping lists and meal prep recipes based on a user’s search query. You will create four shopping lists that inspire diverse and creative meal prep ideas. The complementary lists will focus on items that pair well to create meals, without being strictly tied to the search term. The fourth list will include four meal prep recipes, some of which may introduce new ingredients or concepts to the user. These recipes should showcase how to create affordable, easy, and tasty meals that are batch-friendly and surprising, helping users discover new ways to use familiar or unfamiliar ingredients.

    Shopping Lists:

    - Substitute items: Alternatives to the user’s search term.
    - Complementary product group 1: Items that pair well with the search term or aid in meal prep, focusing on one product group.
    - Complementary product group 2: Another set of items that pair well with the search term or aid in meal prep, focusing on a different product group.
    - Meal prep recipes: Four meal prep recipes that may or may not all include the search term. Recipes should introduce new, affordable, and easily accessible ingredients where possible.

    Guidelines for Complementary Lists:

    Complementary lists should group items that make sense for meal prep but don’t have to strictly relate to the search term.
    Examples of complementary/bought-together product groups include:

    - Cooking Ingredients (spices, oils, sauces)
    - Proteins
    - Vegetables and Fruits
    - Grains and Legumes
    - Accompaniments

    But don't constrain yourself with those listed groups above, be creative. Be creative but ensure the items contribute to creating balanced, practical, easy to make, exciting and flavorful meal-prep meals.
    Include an explanation for why the items were chosen, focusing on their role in meal prep.

    Guidelines for Meal Prep Recipes:
    - Recipes should include four meal prep ideas that offer variety in preparation styles, dietary options, and core ingredients.
    - Not all recipes need to include the query—some, it should use ingredients from the other generated lists, and introduce new ingredients or combinations that users may not have thought of before.
    - Focus on low-cost, batch-friendly, and easy-to-make meals that last well for meal prep.
    - Recipes should be surprising yet practical, showing users how to turn simple, affordable ingredients into flavorful meals.
    - Each recipe must include a reason highlighting its simplicity, versatility, affordability, and how it introduces new meal prep possibilities.
    - Also include an estimated prep time.

    Example:

    "query": "avocado"
    "content": {{
      "a": {{
        "type": "Substitute",
        "title": "Other creamy or rich ingredients",
        "items": ["Hummus", "Greek Yogurt", "Tahini", "Mashed Sweet Potato", "Cottage Cheese"],
        "reason": "These substitutes provide a creamy texture or rich flavor similar to avocado, suitable for a variety of recipes."
      }},
      "b": {{
        "type": "Complementary",
        "title": "Fresh Ingredients for Balanced Dishes",
        "items": ["Cherry Tomatoes", "Cilantro", "Cucumber", "Lime", "Spinach"],
        "reason": "These ingredients pair with avocado to create fresh and balanced meal prep options."
      }},
      "c": {{
        "type": "Complementary",
        "title": "Proteins and Grains for Filling Meals",
        "items": ["Hard-Boiled Eggs", "Canned Black Beans", "Brown Rice", "Chicken Thighs", "Quinoa"],
        "reason": "These proteins and grains add substance and balance to avocado-based meal prep dishes."
      }},
      "d": {{
        "type": "Meal Prep Recipes",
        "title": "Affordable, Discovery-Oriented Meal Prep Recipes",
        "recipes": [
          {{
            "name": "Avocado and Black Bean Grain Bowls",
            "ingredients": ["Quinoa", "Avocado", "Canned Black Beans", "Cherry Tomatoes", "Lime"],
            "reason": "This plant-based bowl is hearty, stores well, and introduces fresh, vibrant flavors with simple ingredients.",
            "prep_time": "30 minutes"
          }},
          {{
            "name": "Sweet Potato and Chickpea Curry",
            "ingredients": ["Sweet Potatoes", "Canned Chickpeas", "Coconut Milk", "Curry Powder", "Spinach"],
            "reason": "A comforting, one-pot dish that is affordable, easy to make, and lasts well in the fridge.",
            "prep_time": "30 minutes"
          }},
          {{
            "name": "Lemon Herb Chicken and Rice",
            "ingredients": ["Chicken Thighs", "Brown Rice", "Garlic", "Fresh Lemons", "Olive Oil"],
            "reason": "A simple yet flavorful meal prep recipe using minimal ingredients that are affordable and filling.",
            "prep_time": "25 minutes"
          }},
          {{
            "name": "Vegetarian Lentil and Avocado Wraps",
            "ingredients": ["Cooked Lentils", "Avocado", "Whole Wheat Wraps", "Cucumber", "Lime"],
            "reason": "A light, vegetarian option that combines creamy avocado with hearty lentils in an easy-to-assemble wrap.",
            "prep_time": "10 minutes"
          }}
        ]
      }}
    }}

    Now, generate shopping lists and meal prep ideas for "{query}" using the same json structure with query and content. The output from this should be the json. Make sure the suggestions are practical, creative, and exciting!
    '''
    return prompt


In [ ]:
def get_chat_response(query, embeddings, product_names):
    # Initialize Cohere client
    try:
        co = cohere.Client(api_key=cohere_api_key)
    except Exception as e:
        return {"error": f"Failed to initialize Cohere client: {str(e)}"}

    # Construct the prompt
    prompt = construct_prompt(query)

    # Start the chat stream
    try:
        stream = co.chat_stream(
            model='command-r-08-2024',
            message=prompt,
            temperature=0.3,
            chat_history=[],
            prompt_truncation='OFF',
        )
    except Exception as e:
        return {"error": f"Chat stream failed: {str(e)}"}

    # Collect the response text
    response = ""
    try:
        for event in stream:
            if event.event_type == "text-generation":
                response += event.text
    except Exception as e:
        return {"error": f"Error reading chat stream: {str(e)}"}

    # Clean and parse the response
    cleaned_output = response.replace("```json", "").replace("```", "").strip()

    try:
        parsed_json = json.loads(cleaned_output)
    except json.JSONDecodeError as e:
        return {"error": f"Error parsing JSON: {str(e)}", "raw_response": cleaned_output}

    # Extract recipes from parsed JSON
    recipes = parsed_json.get('content', {}).get('d', {}).get('recipes', [])

    # Prepare data for the DataFrame
    data = []
    for recipe in recipes:
        recipe_name = recipe.get('name')
        reason = recipe.get('reason')
        ingredients_raw = recipe.get('ingredients', [])

        # Use semantic_search to get matching ingredients
        ingredients = []
        for ingredient in ingredients_raw:
            search_results = semantic_search(ingredient, embeddings, product_names, top_n=1)
            ingredients.extend(search_results)

        # Format ingredients as a comma-separated array with single quotes
        formatted_ingredients = "[" + ", ".join([f"'{ing}'" for ing in ingredients]) + "]"

        # Append to data list
        data.append({
            "Recipe Name": recipe_name,
            "Reason": reason,
            "Ingredients": formatted_ingredients
        })

    # Create the DataFrame
    df = pd.DataFrame(data, columns=["Recipe Name", "Reason", "Ingredients"])

    return df

In [ ]:
answer = get_chat_response("hoisin sauce", name_embeddings, product_names)
answer

,Recipe Name,Reason,Ingredients
0,Hoisin Glazed Pork and Rice Bowls,"A classic combination of Hoisin and pork, serv...","['Stuffed Pork Tenderloin', 'Hoisin Sauce', 'B..."
1,Teriyaki Chicken and Vegetable Stir-Fry,A quick and easy stir-fry using Teriyaki sauce...,"['Chicken Breast Nuggets', 'Teriyaki Sauce', '..."
2,Sweet and Sour Tofu with Pineapple,A vegetarian option with a unique twist. The s...,"['Firm Tofu', 'Sweet and Sour Sauce', 'Pineapp..."
3,Bacon-Wrapped Shrimp with Hoisin Glaze,A fun and indulgent appetizer or main course. ...,"['Chips, Shrimp Big Size', 'Hoisin Sauce', 'Ba..."


In [ ]:
answer = get_chat_response("egg white", name_embeddings, product_names)
answer

,Recipe Name,Reason,Ingredients
0,Egg White and Vegetable Frittata,"A hearty, protein-rich meal that's perfect for...","['Egg Whites', 'Green Peppers', 'Onion', 'Whit..."
1,Tofu and Egg White Stir-Fry,"A quick, flavorful stir-fry that combines the ...","['Firm Tofu', 'Egg Whites', 'Broccoli', 'Carro..."
2,Egg White and Avocado Toast,A simple yet satisfying breakfast or snack opt...,"['Egg Whites', 'Avocado', 'Whole Wheat Bread',..."
3,Zoodle and Egg White Bowl,"A fresh, low-carb meal that's perfect for summ...","['Ready Veggie Zucchini Noodles ', 'Egg Whites..."
